# ANN Models

This notebook builds and trains two Artificial Neural Network (ANN) using `Updated_Metadata_2.csv` dataset. The models are developed using the **same architecture and hyperparameters** but one use EarlyStopping and the other one doesn't.

### Key Steps:

1. **Data Preparation**
   - Loads training and testing datasets.
   - Label encodes the target variable and applies one-hot encoding for neural network compatibility.
   - Features are standardized using `StandardScaler`.

2. **Model Architecture**
   - A deep neural network is defined with 4 hidden layers (500 → 300 → 100 → 45 neurons).
   - Each layer uses ReLU activation, batch normalization, and dropout regularization.
   - The output layer uses softmax activation for multi-class classification.

3. **Model Training**
   - The model is compiled with Adam optimizer and categorical cross-entropy loss.
   - **EarlyStopping** is applied to prevent overfitting in one version of the model.
   - Two versions are trained:
     - **Kaggle-optimized ANN model**: trained without EarlyStopping and tuned for highest Kaggle accuracy.
     - **Generalized ANN model**: uses EarlyStopping for stability and better performance on test data.
     
4. **Evaluation and Saving**
   - Both models are evaluated using test accuracy, balanced accuracy and Classification Report.
   - The models are saved as `ann_best_kaggle.h5` and `ann_best_generalized.h5`.



## Import packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import GlorotUniform, HeNormal
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score, average_precision_score

import random
import os

## ANN for Kaggle Competation Only

### Data Preprocessing 

In [2]:
# Load dataset
training_df = pd.read_csv("Updated_Metadata_2.csv")
testing_df = pd.read_csv("Updated_Metadata_Test_2.csv")

# Encode the target variable
target_column = "activity"
label_encoder = LabelEncoder()
training_df[target_column] = label_encoder.fit_transform(training_df[target_column])
testing_df[target_column] = label_encoder.fit_transform(testing_df[target_column])

# Separate features and target
X_train = training_df.drop(columns=["user_snippet", target_column])  # Exclude non-numeric and target columns
y_train = training_df[target_column]

X_test = testing_df.drop(columns=["user_snippet", target_column])  # Exclude non-numeric and target columns
y_test = testing_df[target_column]

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Model Development

In [3]:
# Get the number of unique classes for output layer
num_classes = len(label_encoder.classes_)

# Convert integer labels to One-Hot Encoding
y_train_one_hot = to_categorical(y_train, num_classes)  # Convert to One-Hot Encoding
y_test_one_hot = to_categorical(y_test, num_classes)  # Convert to One-Hot Encoding

SEED = 999999
np.random.seed(SEED)  # Set NumPy seed
random.seed(SEED)  # Set Python random seed
tf.random.set_seed(SEED)  # Set TensorFlow seed

# Define the model with fixed weight initializers
model = Sequential([
    Dense(500, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(300, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(100, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(45, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    
    Dense(num_classes, activation='softmax', kernel_initializer=GlorotUniform())  # Output layer
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

custom_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001
)

# Train the model
history = model.fit(X_train, y_train_one_hot, 
                    epochs=12, 
                    batch_size=30, 
                    validation_split=0.2, 
                    verbose=1)


# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1/12
169/169 [==============================] - 2s 4ms/step - loss: 0.5619 - accuracy: 0.8025 - val_loss: 1.2959 - val_accuracy: 0.7837
Epoch 2/12
169/169 [==============================] - 1s 4ms/step - loss: 0.2851 - accuracy: 0.9051 - val_loss: 1.5587 - val_accuracy: 0.7702
Epoch 3/12
169/169 [==============================] - 1s 3ms/step - loss: 0.2446 - accuracy: 0.9156 - val_loss: 1.3244 - val_accuracy: 0.7821
Epoch 4/12
169/169 [==============================] - 1s 3ms/step - loss: 0.2069 - accuracy: 0.9328 - val_loss: 1.9158 - val_accuracy: 0.7829
Epoch 5/12
169/169 [==============================] - 1s 3ms/step - loss: 0.1909 - accuracy: 0.9340 - val_loss: 1.6908 - val_accuracy: 0.7845
Epoch 6/12
169/169 [==============================] - 1s 4ms/step - loss: 0.1745 - accuracy: 0.9382 - val_loss: 1.6272 - val_accuracy: 0.7924
Epoch 7/12
169/169 [==============================] - 1s 3ms/step - loss: 0.1676 - accuracy: 0.9431 - val_loss: 1.6852 - val_accuracy: 0.7734
Epoch 

### Save the model

In [4]:
# Save the model
model.save("ann_best_kaggle.h5")
print("Model saved as ann.h5")

Model saved as ann.h5


### Classification Report

In [5]:
from tensorflow.keras.models import load_model

model_best = load_model("ann_best_kaggle.h5")

y_pred_probs_ann = model_best.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs_ann, axis=1)
balanced_accuracy_ann = balanced_accuracy_score(y_test, y_pred_classes)
accuracy_ann = accuracy_score(y_test, y_pred_classes)
report = classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_)

print("ANN Test Balanced Accuracy:", balanced_accuracy_ann)
print("ANN Accuracy:", accuracy_ann)
print("Classification Report:\n", report)

61/61 [==============================] - 0s 942us/step
ANN Test Balanced Accuracy: 0.7550009221331074
ANN Accuracy: 0.8252326783867632
Classification Report:
               precision    recall  f1-score   support

  Downstairs       0.55      0.59      0.57       174
     Jogging       0.94      0.98      0.96       689
     Sitting       0.67      1.00      0.80        22
    Standing       0.93      0.65      0.77        43
    Upstairs       0.64      0.45      0.53       238
     Walking       0.82      0.86      0.84       768

    accuracy                           0.83      1934
   macro avg       0.76      0.76      0.74      1934
weighted avg       0.82      0.83      0.82      1934



### Predict the output for Kaggle dataset

In [6]:
from tensorflow.keras.models import load_model
model_best = load_model("ann_best_kaggle.h5")

In [7]:
# Load dataset
# testing_kaggle_df = pd.read_csv("metadata_kaggle2.csv")
testing_kaggle_df = pd.read_csv("Updated_Metadata_Kaggle_2.csv")
X_test_ID = testing_kaggle_df["user_snippet"]
X_test_kaggle = testing_kaggle_df.drop(columns=["user_snippet"])  # Exclude non-numeric and target columns

# Standardize features
X_test_kaggle_scaled = scaler.transform(X_test_kaggle)

In [8]:
y_pred_probs_ann = model_best.predict(X_test_kaggle_scaled)
y_pred_classes = np.argmax(y_pred_probs_ann, axis=1)
y_pred_original = label_encoder.inverse_transform(y_pred_classes)
prediction_ann = pd.DataFrame({"user_snippet": X_test_ID, "predicted" : y_pred_original})
prediction_ann.to_csv("output_ann_best_kaggle.csv", index=False)
print("Saved the output successfully!")

79/79 [==============================] - 0s 1ms/step
Saved the output successfully!


## More Generalized ANN Model

### Data Preprocessing 

In [9]:
# Load dataset
training_df = pd.read_csv("Updated_Metadata_2.csv")
testing_df = pd.read_csv("Updated_Metadata_Test_2.csv")

# Encode the target variable
target_column = "activity"
label_encoder = LabelEncoder()
training_df[target_column] = label_encoder.fit_transform(training_df[target_column])
testing_df[target_column] = label_encoder.fit_transform(testing_df[target_column])

# Separate features and target
X_train = training_df.drop(columns=["user_snippet", target_column])  # Exclude non-numeric and target columns
y_train = training_df[target_column]

X_test = testing_df.drop(columns=["user_snippet", target_column])  # Exclude non-numeric and target columns
y_test = testing_df[target_column]

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Model Development

In [10]:
# Get the number of unique classes for output layer
num_classes = len(label_encoder.classes_)

# Convert integer labels to One-Hot Encoding
y_train_one_hot = to_categorical(y_train, num_classes)  # Convert to One-Hot Encoding
y_test_one_hot = to_categorical(y_test, num_classes)  # Convert to One-Hot Encoding

SEED = 999999
np.random.seed(SEED)  # Set NumPy seed
random.seed(SEED)  # Set Python random seed
tf.random.set_seed(SEED)  # Set TensorFlow seed

# Define the model with fixed weight initializers
model = Sequential([
    Dense(500, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(300, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(100, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(45, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    
    Dense(num_classes, activation='softmax', kernel_initializer=GlorotUniform())  # Output layer
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

custom_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001
)

# Train the model
history = model.fit(X_train, y_train_one_hot, 
                    epochs=100, 
                    batch_size=30, 
                    validation_split=0.2, 
                    verbose=1, 
                    callbacks=[custom_early_stopping])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1/100
169/169 [==============================] - 2s 4ms/step - loss: 0.5619 - accuracy: 0.8025 - val_loss: 1.2959 - val_accuracy: 0.7837
Epoch 2/100
169/169 [==============================] - 1s 3ms/step - loss: 0.2851 - accuracy: 0.9051 - val_loss: 1.5587 - val_accuracy: 0.7702
Epoch 3/100
169/169 [==============================] - 1s 3ms/step - loss: 0.2446 - accuracy: 0.9156 - val_loss: 1.3244 - val_accuracy: 0.7821
Epoch 4/100
169/169 [==============================] - 1s 3ms/step - loss: 0.2069 - accuracy: 0.9328 - val_loss: 1.9158 - val_accuracy: 0.7829
Epoch 5/100
169/169 [==============================] - 0s 3ms/step - loss: 0.1909 - accuracy: 0.9340 - val_loss: 1.6908 - val_accuracy: 0.7845
Epoch 6/100
169/169 [==============================] - 1s 3ms/step - loss: 0.1745 - accuracy: 0.9382 - val_loss: 1.6272 - val_accuracy: 0.7924
Epoch 7/100
169/169 [==============================] - 1s 3ms/step - loss: 0.1676 - accuracy: 0.9431 - val_loss: 1.6852 - val_accuracy: 0.7734

### Save the model

In [11]:
# Save the model
model.save("ann_best_generalized.h5")
print("Model saved as ann.h5")

Model saved as ann.h5


### Classification Report

In [12]:
from tensorflow.keras.models import load_model

model_best = load_model("ann_best_generalized.h5")

y_pred_probs_ann = model_best.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs_ann, axis=1)
balanced_accuracy_ann = balanced_accuracy_score(y_test, y_pred_classes)
accuracy_ann = accuracy_score(y_test, y_pred_classes)
report = classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_)

print("ANN Test Balanced Accuracy:", balanced_accuracy_ann)
print("ANN Accuracy:", accuracy_ann)
print("Classification Report:\n", report)

61/61 [==============================] - 0s 955us/step
ANN Test Balanced Accuracy: 0.8076698930227194
ANN Accuracy: 0.8531540847983454
Classification Report:
               precision    recall  f1-score   support

  Downstairs       0.57      0.70      0.63       174
     Jogging       0.93      0.98      0.96       689
     Sitting       0.67      1.00      0.80        22
    Standing       0.94      0.70      0.80        43
    Upstairs       0.71      0.62      0.67       238
     Walking       0.90      0.85      0.88       768

    accuracy                           0.85      1934
   macro avg       0.79      0.81      0.79      1934
weighted avg       0.86      0.85      0.85      1934



### Predict the output for Kaggle dataset

In [13]:
from tensorflow.keras.models import load_model
model_best = load_model("ann_best_generalized.h5")

In [14]:
# Load dataset
# testing_kaggle_df = pd.read_csv("metadata_kaggle2.csv")
testing_kaggle_df = pd.read_csv("Updated_Metadata_Kaggle_2.csv")
X_test_ID = testing_kaggle_df["user_snippet"]
X_test_kaggle = testing_kaggle_df.drop(columns=["user_snippet"])  # Exclude non-numeric and target columns

# Standardize features
X_test_kaggle_scaled = scaler.transform(X_test_kaggle)

In [15]:
y_pred_probs_ann = model_best.predict(X_test_kaggle_scaled)
y_pred_classes = np.argmax(y_pred_probs_ann, axis=1)
y_pred_original = label_encoder.inverse_transform(y_pred_classes)
prediction_ann = pd.DataFrame({"user_snippet": X_test_ID, "predicted" : y_pred_original})
prediction_ann.to_csv("output_ann_best_generalized.csv", index=False)
print("Saved the output successfully!")

79/79 [==============================] - 0s 915us/step
Saved the output successfully!
